In [1]:
import diamond_music_utils as dmu

from importlib import reload
reload(dmu)
import numpy as np
import pprint
pp = pprint.PrettyPrinter(sort_dicts=False)
from fractions import Fraction
import copy
import logging
from numpy.random import default_rng
rng = np.random.default_rng()
CSD_FILE = 'ball3.csd'
LOGNAME = 'ball3.log'
WAV_OUT = '/home/prent/Music/sflib/ball3.wav'
CON_OUT = '/home/prent/Music/sflib/ball3-t'

In [2]:
csound_running = False

In [3]:
all_ratios = np.array(dmu.build_all_ratios())
ratio_strings = dmu.build_ratio_strings(all_ratios) # create a (16,16) array of ratios in string form. Change those that are '1' to '1/1'
print(f'{all_ratios.shape = }')
# print the ratios as fractions
limit = 31
end_denom = limit + 1
start_denom = (end_denom) // 2
o_numerator = np.arange(start_denom, end_denom, 1) # create a list of overtones
u_denominator = np.arange(start_denom, end_denom, 1) # create a list of undertones
print(f'Tonality Diamond in the Cassandra orientation to the {limit}-limit', end='')
i = 0
for oton_root in u_denominator:
    print()
    for overtone in o_numerator:
        print(f'{str(Fraction(all_ratios[i]).limit_denominator())}', end = '\t')
        i += 1
all_ratio_strings = ratio_strings.reshape(256,)

all_ratios.shape = (256,)
Tonality Diamond in the Cassandra orientation to the 31-limit
1	17/16	9/8	19/16	5/4	21/16	11/8	23/16	3/2	25/16	13/8	27/16	7/4	29/16	15/8	31/16	
32/17	1	18/17	19/17	20/17	21/17	22/17	23/17	24/17	25/17	26/17	27/17	28/17	29/17	30/17	31/17	
16/9	17/9	1	19/18	10/9	7/6	11/9	23/18	4/3	25/18	13/9	3/2	14/9	29/18	5/3	31/18	
32/19	34/19	36/19	1	20/19	21/19	22/19	23/19	24/19	25/19	26/19	27/19	28/19	29/19	30/19	31/19	
8/5	17/10	9/5	19/10	1	21/20	11/10	23/20	6/5	5/4	13/10	27/20	7/5	29/20	3/2	31/20	
32/21	34/21	12/7	38/21	40/21	1	22/21	23/21	8/7	25/21	26/21	9/7	4/3	29/21	10/7	31/21	
16/11	17/11	18/11	19/11	20/11	21/11	1	23/22	12/11	25/22	13/11	27/22	14/11	29/22	15/11	31/22	
32/23	34/23	36/23	38/23	40/23	42/23	44/23	1	24/23	25/23	26/23	27/23	28/23	29/23	30/23	31/23	
4/3	17/12	3/2	19/12	5/3	7/4	11/6	23/12	1	25/24	13/12	9/8	7/6	29/24	5/4	31/24	
32/25	34/25	36/25	38/25	8/5	42/25	44/25	46/25	48/25	1	26/25	27/25	28/25	29/25	6/5	31/25	
16/13	17/13	18/13	19/13	20/13	21/13	22/13	23/1

## Make a multivoice piece
This is the first attempt at composing a multivoice piece where every note is written out. Later I can automate the process of choosing variables

In [4]:
#
# Start up an instance of csound
#
#
def start_csound():
    global csound_running
    if not csound_running: 
        dmu.start_logger(LOGNAME)
        csd_content, lines = dmu.load_csd(CSD_FILE)
        logging.info(f'Starting to log to {LOGNAME}')
        logging.info(f'Loaded the csd file {CSD_FILE}. There are {lines} lines read containg {len(csd_content)} bytes')
        cs, pt = dmu.load_csound(csd_content)
        csound_running = True
        return(cs, pt)
    else: 
        print(f'csound is already running')
        return(None, None)
#
#
# stop the csound instance
#
#
def stop_csound(cs, pt):
    global csound_running
    if csound_running:
        dmu.printMessages(cs)
        pt.stop() # this is important I think. It closes the output file.
        pt.join()   
        dmu.printMessages(cs)    
        cs.reset()
        csound_running = False
    else:
        print(f"csound isn't running")
    return (None)

In [5]:
# Is there anything else I need to track about the voice? If not, this could be simplified
# track the name, the start time, and the instrument number
voice_time = {'fpn': {'full': 'finger piano', 'start': 0, 'number': 1},
      'bnp': {'full': 'bass finger piano', 'start': 0, 'number': 2},
      'bdl': {'full': 'bass balloon drum', 'start': 0, 'number': 3},
      'bdm': {'full': 'medium balloon drum', 'start': 0, 'number': 4},
      'bdh': {'full': 'high balloon drum', 'start': 0, 'number': 5},
      'bfl': {'full': 'bass flute', 'start': 0, 'number': 6},
      'obo': {'full': 'oboe', 'start': 0, 'number': 7},
      'cla': {'full': 'clarinet', 'start': 0, 'number': 8},
      'bss': {'full': 'bassoon', 'start': 0, 'number': 9},
      'frn': {'full': 'french horn', 'start': 0, 'number': 10}}

pp.pprint(voice_time)
print(f'{voice_time["fpn"]["full"] = }')
print(f'{voice_time["fpn"]["start"] = }')
print(f'{voice_time["fpn"]["number"] = }')

{'fpn': {'full': 'finger piano', 'start': 0, 'number': 1},
 'bnp': {'full': 'bass finger piano', 'start': 0, 'number': 2},
 'bdl': {'full': 'bass balloon drum', 'start': 0, 'number': 3},
 'bdm': {'full': 'medium balloon drum', 'start': 0, 'number': 4},
 'bdh': {'full': 'high balloon drum', 'start': 0, 'number': 5},
 'bfl': {'full': 'bass flute', 'start': 0, 'number': 6},
 'obo': {'full': 'oboe', 'start': 0, 'number': 7},
 'cla': {'full': 'clarinet', 'start': 0, 'number': 8},
 'bss': {'full': 'bassoon', 'start': 0, 'number': 9},
 'frn': {'full': 'french horn', 'start': 0, 'number': 10}}
voice_time["fpn"]["full"] = 'finger piano'
voice_time["fpn"]["start"] = 0
voice_time["fpn"]["number"] = 1


In [6]:
# best combination of rank and inversion to minimize the distances between notes in a chord for ranks A & B
# these were found by systematically testing every possible combination and finding those that were closer than 0.7 and 1.3 ratio from each other
best_rank_inversion_combos = np.array([["A", "A", 1, 2],["A", "A", 1, 4],["A", "A", 2, 1],["A", "A", 2, 3],["A", "A", 3, 2],
    ["A", "A", 3, 4],["A", "A", 4, 1],["A", "A", 4, 3],["A", "B", 1, 1],["A", "B", 1, 4],["A", "B", 2, 1],["A", "B", 2, 2],
    ["A", "B", 3, 2],["A", "B", 3, 3],["A", "B", 4, 3],["A", "B", 4, 4],["B", "A", 1, 1],["B", "A", 1, 2],["B", "A", 2, 2],
    ["B", "A", 2, 3],["B", "A", 3, 3],["B", "A", 3, 4],["B", "A", 4, 1],["B", "A", 4, 4],["B", "B", 1, 2],["B", "B", 1, 4],
    ["B", "B", 2, 1],["B", "B", 2, 3],["B", "B", 3, 2],["B", "B", 3, 4],["B", "B", 4, 1],["B", "B", 4, 3],])

# combination of trills_8_step and envelope to convert an 8 step trill into fewer steps
# envelope #15: 1 up, 1 down, 1 up is 1/6 of the hold value
# 14: 2 down & up 1/4 of the hold value
# 13: 4 down & up 1/2 of the hold value
# 12: 5 down & up 2/3 of the hold value
# 11: 6 down & up 3/4 of the hold value
# 1:  8 down & up full hold value
# 2: exponential decay for a short chop
# 8: fast decay for bass finger piano punch
combo_trills = np.array([[15, 4, 20], [14, 4, 16], [13, 4, 8], [12, 4, 6], [11, 4, 5], [1, 4, 4]])

In [72]:
# need a systematic search of all the slides to minimize the maximum distance, defined as more than 1.3 or less than 0.75
# vary the inversions of the two chords.
# def build_chords(mode, ratio, rank, inversion):

reload(dmu)
dmu.init_stored_gliss() 
stored_gliss = dmu.init_stored_gliss() 
bridge_chords = {0: {0: {'mode': 'oton', 'ratio': '16/9', 'rank': 'A'}, 
                     1: {'mode': 'oton', 'ratio': '8/7', 'rank': 'A'}},
                 1: {0: {'mode': 'oton', 'ratio': '8/7', 'rank': 'A'}, 
                     1: {'mode': 'uton', 'ratio': '9/8', 'rank': 'A'}},
                 2: {0: {'mode': 'uton', 'ratio': '9/8', 'rank': 'A'}, 
                     1: {'mode': 'oton', 'ratio': '16/15', 'rank': 'A'}},
                 3: {0: {'mode': 'oton', 'ratio': '16/15', 'rank': 'A'},
                     1: {'mode': 'uton', 'ratio': '1/1', 'rank': 'A'}},
                 4: {0: {'mode': 'uton', 'ratio': '1/1', 'rank': 'A'}, 
                     1: {'mode': 'oton', 'ratio': '1/1', 'rank': 'A'}},
                 5: {0: {'mode': 'oton', 'ratio': '1/1', 'rank': 'A'}, 
                     1: {'mode': 'uton', 'ratio': '7/4', 'rank': 'A'}},
                 6: {0: {'mode': 'uton', 'ratio': '7/4', 'rank': 'A'}, 
                     1: {'mode': 'oton', 'ratio': '16/9', 'rank': 'A'}},
                 7: {0: {'mode': 'uton', 'ratio': '15/8', 'rank': 'A'},
                     1: {'mode': 'oton', 'ratio': '16/9', 'rank': 'A'}},
                 8: {0: {'mode': 'oton', 'ratio': '16/9', 'rank': 'A'},
                     1: {'mode': 'uton', 'ratio': '15/8', 'rank': 'A'}},
                    }   
# pp.pprint(bridge_chords)                 

In [65]:
# this function is only used to evaluate a set of inversions chosen randomly. It takes a while. 2 minutes or so.
def evaluate_voice(one_voice_array):
    sum_of_cents_voice = 0
    # print(f'{one_voice_array}')
    prev_note = one_voice_array[0]
    max_cents_per_voice = 0.0
    total_f_steps = 0
    first_note = True
    for current_note in one_voice_array: # for each note one at a time
        if first_note: # we skip the first one, since it is not sliding from anywhere.
            first_note = False
        else:
            ratio_distance = dmu.ratio_distance(all_ratio_strings[prev_note],all_ratio_strings[current_note])
            max_cents_per_voice = np.max((max_cents_per_voice,abs(ratio_to_cents(ratio_distance))))
            sum_of_cents_voice += abs(ratio_to_cents(ratio_distance))
            prev_note = current_note
    return (max_cents_per_voice, sum_of_cents_voice)

# This cell does a grid search over 25,000 random sets of inversions and tests to find the lowest cent diversion. 
# I found 28 with low cent diversion.
test_cases = 25_000
all_chords_all_inversions = np.zeros((test_cases, 9, 4), dtype = int)
results = []
for inx in np.arange(test_cases):
    all_chords_all_inversions[inx] = np.array([dmu.build_chords(bridge_chords[chords][0]["mode"], bridge_chords[chords][0]["ratio"], \
                                                 bridge_chords[chords][0]["rank"], rng.choice([1,2,3,4])) for chords in bridge_chords])
    all_max_cents = 0
    all_sum_cents = 0
    for voice in all_chords_all_inversions[inx].T:
        # print(f'{voice = }')
        max_cents, sum_of_cents = evaluate_voice(voice)
        all_max_cents = np.max((max_cents, all_max_cents))
        all_sum_cents += sum_of_cents
    results.append([inx, all_max_cents, all_sum_cents])

print(f'{all_chords_all_inversions.shape = }')    
print(f'inx\tmax\tsum cents')    

all_min_cents = 99_999
best_test_cases = []
print(f'examining the results. Here are some at the thousand mark: ')

for inx in np.arange(test_cases):
    if inx % 1000 == 0: print(f'inx: {results[inx][0]}\t{results[inx][1]}\t{round(results[inx][2],0)}')
    if results[inx][1] < 323:
        all_min_cents = np.min((all_min_cents, results[inx][1]))
        best_test_cases.append(inx)
        
print(f'completed analyzing {test_cases = }')
print(f'{all_min_cents = }, {best_test_cases = }')
for inx in np.arange(len(best_test_cases)):
    print(f'{best_test_cases[inx] = }')
    print(f'{results[best_test_cases[inx]][1] = }')
    print(f'{all_chords_all_inversions[best_test_cases][inx].T}')

best_bridge_chord_array = np.array([all_chords_all_inversions[best_test_cases][inx].T for inx in np.arange(len(best_test_cases))])
print(f'{best_bridge_chord_array.shape = }')
inx = 0
for array_chord in best_bridge_chord_array:
    print(f'\nindex into best_bridge_chord_array {inx}:', end='\t')
    v = 0
    for voice in array_chord:
        print(f'\n{v}: ', end='\t')
        v += 1
        for note in voice:
            print(f'{note}', end = ', ')
    inx += 1

In [7]:
# select a random combination of ranks and intervals and return them as an array
def choose_best_rank_inversion_combos():
    return(best_rank_inversion_combos[rng.integers(low = 0, high = best_rank_inversion_combos.shape[0])])
def choose_combo_trills():
    return(combo_trills[rng.integers(low = 0, high = combo_trills.shape[0])])
def choose_valid_rank():
    # return a valid rank
    return rng.choice([rank for rank in dmu.show_inversions().keys()])
def choose_valid_mode(rank):
    return(rng.choice([mode for mode in dmu.show_scales()[rank].keys()]))

In [8]:
# this makes one note with all 15 features - it no longer updates voice_time with new "start" values
def instrument(tone, octave, duration, hold, gliss, voice_name, velocity, stereo, env, upsample, renv, gliss2, gliss3, volume):
    voice = voice_time[voice_name]["number"] # turn the short name into a voice number that csound needs
    # load an array with everything csound needs to create a note - the instrument number expected by the csound file has to be 1
    next_note = np.array((1, duration, hold, velocity, tone, octave, voice, stereo, env, gliss, upsample, renv, gliss2, gliss3, volume))
    return next_note

In [9]:
# convert ratio to cents
def ratio_to_cents(ratio):
    return round(1200 * np.log(ratio)/np.log(2),1)

root = '16/9'
ratio_decimal = dmu.ratio_string_to_float(root)
print(f'{ratio_to_cents(ratio_decimal) = }')
ratio_decimal = 1.2
print(f'{ratio_to_cents(ratio_decimal) = }')

ratio_to_cents(ratio_decimal) = 996.1
ratio_to_cents(ratio_decimal) = 315.6


In [10]:
def show_voice_time_short_name(number):
    for voice in voice_time:
        # print(f'{voice = }, {voice_time[voice]["number"] = }, {voice_time[voice]["start"] = }')
        if voice_time[voice]["number"] == number:
            return (voice)

In [11]:
def init_voice_start_times():
    for instrument in voice_time:
        voice_time[instrument]["start"] = 0

In [12]:
def fix_start_times(note_array):
    # first I need to obtain the keys to the voice_time array by the value in the note_array
    # winds.shape = (135, 15) # array is 135 rows, each one a complete note by 15 columns for the features
    note_num = 0
    start_col = 1
    dur_col = 1
    hold_col = 2
    print(f'{note_array.shape = }')
    for note in note_array:
        voice_name = show_voice_time_short_name(note[6]) # sequential search by number for a name
        current_time = voice_time[voice_name]["start"]
        voice_time[voice_name]["start"] += note[start_col] # this is the missing piece. must happen even if the note doesn't 
        if note[2] > 0.0: # if the hold time is not zero
            note_array[note_num,dur_col] = current_time  # overwrite column 1 (duration) with the new start_time
            note_num += 1 # this should only happen if the hold value is non-zero.
    return note_array[:note_num,:]

In [13]:
# pass a set of parameters and get back an array of notes
def chord_play(note_dict):
    # Build a set of notes                mode               root                 A or B                and inversion
    tones_1 = dmu.build_chords(note_dict["mode"], note_dict["root"], rng.choice(["A","B"]), rng.choice([1,2,3,4])) # get four notes 
    octave = dmu.build_scale_mask(tones_1) + note_dict["octv"] # get the octave mask
    # The number of notes may not be the same as the number of instruments. We may need to duplicate some notes to reach the number of instruments
    # num_inst = note_dict["instruments"].shape[0]
    num_inst = note_dict["instruments"].shape[0]
    num_notes = tones_1.shape[0]
    while num_notes < num_inst:
        # print(f'{num_inst = }, {num_notes = }, {tones_1.shape = }')
        # copy the duplicate the first element (0) to be the last element until the number of notes = number of instruments
        tones_1 = np.append(tones_1, tones_1[0], axis = None)
        octave = np.append(octave, octave[0], axis = None)
        num_notes = tones_1.shape[0]
    if num_inst < num_notes:
        print(f'Not enough instruments specified in {note_dict["instruments"] = }, {num_notes = }')
        return None
    
    dur = np.ones((num_inst,), dtype = float) * note_dict["dur"] # all the notes have the same duration
    hold = np.ones((num_inst,), dtype = float) * note_dict["hold"] # how long to hold them
    env = np.ones((num_inst,), dtype = float) * note_dict["env"] # what envelope to use for all of them
    renv = np.ones((num_inst,), dtype = float) * note_dict["renv"] # what should the right envelope be
    instruments = np.empty((num_inst * note_dict["repeats"], 15), dtype = float)
    print(f'{instruments.shape = }')
    
    iterations = 0
    for repeats in np.arange((note_dict["repeats"])):
        for inst in np.arange(num_inst):
            # spread the notes across the different instruments
            instruments[iterations] = instrument(tones_1[inst], octave[inst], dur[inst], hold[inst], note_dict["defaults"][0], \
                        note_dict["instruments"][inst], note_dict["defaults"][1], note_dict["defaults"][2], env[inst], \
                        note_dict["defaults"][3], renv[inst], note_dict["defaults"][4], note_dict["defaults"][5], \
                        note_dict["defaults"][6])
            iterations += 1
    print(f'{instruments.shape = }')
    return instruments

In [14]:
# pass a set of parameters and get back an array of notes
def scale_arpeggio(note_dict):
              
    # set of notes from a scale based on  mode            root             rank 
    tones_1 = dmu.build_scales(note_dict["mode"], note_dict["root"], note_dict["rank"]) # get 8 notes 
    tones_1 = np.append(tones_1, tones_1[0]) # add the first note to the end
    
    if note_dict["roll"] > 0:
        tones_1 = np.roll(tones_1, note_dict["roll"])
    
    octave = dmu.build_scale_mask(tones_1) # get the mask for this 9 tone scale, which might have been rolled
    
    if note_dict["flip"]: # reverse the scale and the octave mask
        tones_1 = np.flip(tones_1)
        octave = np.flip(octave)
    
    octave += note_dict["octv"] # add the requested octave value to the mask
    
    num_inst = note_dict["instruments"].shape[0]
    num_notes = tones_1.shape[0]
    
    dur = note_dict["dur"] # each instrument has its own duration
    hold = note_dict["hold"] # each instrument has its own hold value
    env = np.ones((num_inst,), dtype = float) * note_dict["env"] # what envelope to use for instruments
    renv = np.ones((num_inst,), dtype = float) * note_dict["renv"] # right envelope 
    instruments = np.empty((num_notes * num_inst, 15), dtype = float)
    
    iterations = 0
    for (note, octv) in zip(tones_1,octave): # iterate a match between a note and it's octave
        for inst in np.arange(num_inst):
            instruments[iterations] = instrument(note, octv, dur[inst], hold[inst], note_dict["defaults"][0], \
                        note_dict["instruments"][inst], note_dict["defaults"][1], note_dict["defaults"][2], env[inst], \
                        note_dict["defaults"][3], renv[inst], note_dict["defaults"][4], note_dict["defaults"][5], \
                        note_dict["defaults"][6])
            iterations += 1
    return instruments[:iterations,:]

In [15]:
# pass a set of parameters and get back an array of notes
def hexad_arpeggio(note_dict):
    # Build a set of notes                mode               root         rank           and random inversion
    tones_1 = dmu.build_chords(note_dict["mode"], note_dict["root"], note_dict["rank"], rng.choice([1,2,3,4])) # get four notes 
    octave = dmu.build_scale_mask(tones_1) + note_dict["octv"] # get the octave mask and add it to the passed octave
    
    if note_dict["flip"]: # reverse the scale and the octave mask
        tones_1 = np.flip(tones_1)
        octave = np.flip(octave)
    
    num_inst = note_dict["instruments"].shape[0]
    num_notes = tones_1.shape[0]
    dur = note_dict["dur"]   # each instrument can have it's own duration, the same for all repeats. But things get complicated if they don't
    hold = note_dict["hold"] # each instrument can have it's own hold value, repeated each repeat
    env = np.ones((num_inst,), dtype = float) * note_dict["env"] # same envelope for all of the instruments
    renv = np.ones((num_inst,), dtype = float) * note_dict["renv"] # same envelope for all of the instruments
    instruments = np.empty((num_notes * num_inst * note_dict["repeats"], 15), dtype = float)
    mix_mask = rng.choice(2, size=(instruments.shape[0]), p = [1 - note_dict["density"], note_dict["density"]])
    # print(f'{np.sum(mix_mask) = }')
    # print(f'{mix_mask.shape = }')
    iterations = 0
    for repeats in np.arange(note_dict["repeats"]): 
        roll = rng.choice([2, 0, -2])
        tones_1 = np.roll(tones_1, roll)
        for (note, octv) in zip(tones_1,octave): # iterate a match between a note and it's octave
            for inst in np.arange(num_inst):
                instruments[iterations] = instrument(note, octv, dur[inst], hold[inst] * mix_mask[iterations], note_dict["defaults"][0], \
                        note_dict["instruments"][inst], note_dict["defaults"][1], note_dict["defaults"][2], env[inst], \
                        note_dict["defaults"][3], renv[inst], note_dict["defaults"][4], note_dict["defaults"][5], \
                        note_dict["defaults"][6])
                iterations += 1
    return instruments

In [16]:
# pass a set of parameters and get back an array of notes
def root_chord_slide(note_dict):
    # Build a set of notes 
    tones_1 = dmu.build_chords(note_dict["mode"], note_dict["root"], note_dict["combo"][0], int(note_dict["combo"][2])) # get four notes 
    tones_2 = dmu.build_chords(note_dict["mode"], note_dict["root"], note_dict["combo"][1], int(note_dict["combo"][3])) # get another four notes 
    octave = dmu.build_scale_mask(tones_1) + note_dict["octv"] # get the octave mask
    # print(f'{note_dict["octv"] = }')
    # build_slides returns an ftable number for each note in the chord and updates global stored_gliss table
    
    gliss = dmu.build_slides(tones_1, tones_2, gliss_type = note_dict["gliss_type"]) 
    # The number of notes may not be the same as the number of instruments. We may need to duplicate some notes to reach the number of instruments
    num_inst = note_dict["instruments"].shape[0]
    num_notes = tones_1.shape[0]
    while num_notes < num_inst:
        # print(f'{num_inst = }, {num_notes = }, {tones_1.shape = }')
        # copy the duplicate the first element (0) to be the last element until the number of notes = number of instruments
        tones_1 = np.append(tones_1, tones_1[0], axis = None)
        octave = np.append(octave, octave[0], axis = None)
        gliss = np.append(gliss, gliss[0], axis = None)
        num_notes = tones_1.shape[0]
    if num_inst < num_notes:
        print(f'Not enough instruments specified in {note_dict["instruments"] = }, {num_notes = }')
        return None
    
    dur = np.ones((num_inst,), dtype = float) * note_dict["dur"] # all the notes have the same duration
    # print(f'in chord_slides. {note_dict["hold"] = }')
    hold = np.ones((num_inst,), dtype = float) * note_dict["hold"] # how long to hold them
    # print(f'in chord_slides. {hold = }')
    env = np.ones((num_inst,), dtype = float) * note_dict["env"] # what envelope to use for all of them
    renv = np.ones((num_inst,), dtype = float) * note_dict["renv"] # what should the right envelope be
    
    iterations = 0
    instruments = np.empty((num_inst * note_dict["repeats"], 15), dtype = float)
    for repeats in np.arange((note_dict["repeats"])):
        for inst in np.arange(num_inst):
            # instruments[:,inx] = instrument(tones_1[inx], octave[inx], dur[inx], hold[inx], gliss[inx], \
            #                     note_dict["instruments"][inx], env = env[inx], renv = renv[inx])
            
            instruments[iterations] = instrument(tones_1[inst], octave[inst], dur[inst], hold[inst], gliss[inst], \
                        note_dict["instruments"][inst], note_dict["defaults"][1], note_dict["defaults"][2], env[inst], \
                        note_dict["defaults"][3], renv[inst], note_dict["defaults"][4], note_dict["defaults"][5], \
                        note_dict["defaults"][6])
            iterations += 1
            
    # print(f'{instruments.shape = }')
    return instruments

In [17]:
# slide bewteen any two arbitrary chords definded by mode, root, rank, and inversion
def chord_slide(note_dict):
    # Build a set of notes 
    tones_1 = dmu.build_chords(note_dict["mode1"], note_dict["root1"], note_dict["rank1"], note_dict["inv_1"] ) # get four notes 
    tones_2 = dmu.build_chords(note_dict["mode2"], note_dict["root2"], note_dict["rank2"], note_dict["inv_2"] ) # get another four notes 
    
    # octave = dmu.build_scale_mask(tones_1) + note_dict["octv"] # get the octave mask for the starting chord. This defeats the purpose of controlling the slides cents
    octv = note_dict["octv"] # if it's an array, then iterate over it, after setting duration to zero for the first iterations, then the final iteration gets the duration value
    repeats = octv.shape[0]
    gliss = dmu.build_slides(tones_1, tones_2, gliss_type = note_dict["gliss_type"]) # returns a list of glissando f table numbers, not the whole f table
    # The number of notes may not be the same as the number of instruments. We may need to duplicate some notes to reach the number of instruments
    num_inst = note_dict["instruments"].shape[0]
    num_notes = tones_1.shape[0]
    while num_notes < num_inst:
        # print(f'{num_inst = }, {num_notes = }, {tones_1.shape = }')
        # copy the duplicate the first element (0) to be the last element until the number of notes = number of instruments
        tones_1 = np.append(tones_1, tones_1[0], axis = None)
        # octave = np.append(octave, octave[0], axis = None)
        gliss = np.append(gliss, gliss[0], axis = None)
        num_notes = tones_1.shape[0]
    if num_inst < num_notes:
        print(f'Not enough instruments specified in {note_dict["instruments"] = }, {num_notes = }')
        return None
    
    dur = np.ones((repeats,), dtype = float) * note_dict["dur"] # all the notes have the same duration 
    dur[:-1] = 0 # set the first iterations to 0 duration 
    # print(f'dur should be all zeros until the last iteration: {dur = }')
    hold = np.ones((num_inst,), dtype = float) * note_dict["hold"] # how long to hold them
    env = np.ones((num_inst,), dtype = float) * note_dict["env"] # what envelope to use for all of them
    renv = np.ones((num_inst,), dtype = float) * note_dict["renv"] # what should the right envelope be
    # Ins Sta Dur Vel Ton Oct Voi Ste En1 Gls Ups Ren 2gl  3gl Vol
    # [1,  0, 4,  60, 32,  2, 7,  8,  1, 800, 1,  1,  799, 799, 30]	start: 0.0,	dur: 4.4
    # the goal is to play notes in each instrument with duration = 0 until you play the last octave, then set duration to 4
    iterations = 0
    instruments = np.empty((num_inst * repeats, 15), dtype = float)
    # print(f'{note_dict["mode1"]}, {note_dict["root1"]}, {note_dict["rank1"]}, {note_dict["inv_1"]} '
    #      f'{note_dict["mode2"]}, {note_dict["root2"]}, {note_dict["rank2"]}, {note_dict["inv_2"]}')
    
    for inst in np.arange(num_inst): # once for each instrument
        for octave in octv: # number of repeats = number of elements in the octv array
            current_octave = iterations % repeats
            # print(f'{inst = }, {octave = }, {dur[current_octave] = } ')
            instruments[iterations] = instrument(tones_1[inst], octave, dur[current_octave], hold[inst], gliss[inst], \
                        note_dict["instruments"][inst], note_dict["defaults"][1], note_dict["defaults"][2], env[inst], \
                        note_dict["defaults"][3], renv[inst], note_dict["defaults"][4], note_dict["defaults"][5], \
                        note_dict["defaults"][6])
            iterations += 1
            
    # print(f'{instruments.shape = }')
    return instruments

In [128]:
# print the 4 inversions of the hexad through the bridge chords
# since we have the ability to select a different inversion for each of the chords in the bridge chord changes,
# we want to pick those inversions that minimize the slide distance between notes
temp_array = np.zeros((9,4), dtype = int)
for inversions in np.arange(1,5,1): # 1,2,3,4
    print(f'{inversions = }')
    inx = 0
    for chords in bridge_chords:
        temp_array[inx] = dmu.build_chords(bridge_chords[chords][0]["mode"],bridge_chords[chords][0]["ratio"],bridge_chords[chords][0]["rank"], inversions)
        inx += 1
    print(f'{temp_array.shape}, {temp_array.T} {temp_array[0,:] = }')

inversions = 1
(9, 4), [[ 32 192 194 224 192   0 204 206  32]
 [ 36 196 130 228 128   4 140 142  36]
 [ 40 200  66 232  64   8  76  78  40]
 [ 44 204   2 236   0  12  12  14  44]] temp_array[0,:] = array([32, 36, 40, 44])
inversions = 2
(9, 4), [[ 36 196   2 228   0   4  12  14  36]
 [ 40 200 194 232 192   8 204 206  40]
 [ 44 204 130 236 128  12 140 142  44]
 [ 32 192  66 224  64   0  76  78  32]] temp_array[0,:] = array([36, 40, 44, 32])
inversions = 3
(9, 4), [[ 40 200  66 232  64   8  76  78  40]
 [ 44 204   2 236   0  12  12  14  44]
 [ 32 192 194 224 192   0 204 206  32]
 [ 36 196 130 228 128   4 140 142  36]] temp_array[0,:] = array([40, 44, 32, 36])
inversions = 4
(9, 4), [[ 44 204 130 236 128  12 140 142  44]
 [ 32 192  66 224  64   0  76  78  32]
 [ 36 196   2 228   0   4  12  14  36]
 [ 40 200 194 232 192   8 204 206  40]] temp_array[0,:] = array([44, 32, 36, 40])


In [205]:
 # build a function table that will create a slide for one note in a long set of glissandi passing through nine chords
def build_voice_slide(t_num, one_voice_array):
    
    start_ratio = 1.0
    # print(f'{one_voice_array.shape = }') # should be 9 notes long
    each_f_step = round(1024 / (one_voice_array.shape[0] - 1),0) # 128 for 8 
    # print(f'{each_f_step = }')
    
    fn_array = np.array([t_num, 0.0, 2048.0, -7, start_ratio])
    sum_of_cents_voice = 0
    # print(f'{t_num = }\n{one_voice_array}')
    # print(f'{dmu.show_scale_ratios(one_voice_array)}')
    prev_note = one_voice_array[0]
    max_cents_per_voice = 0.0
    total_f_steps = 0
    first_note = True
    for current_note in one_voice_array: # for each note one at a time
        if first_note: # we skip the first one, since it is not sliding from anywhere.
            first_note = False
        else:
            ratio_distance = dmu.ratio_distance(all_ratio_strings[prev_note],all_ratio_strings[current_note])
            max_cents_per_voice = np.max((max_cents_per_voice, abs(ratio_to_cents(ratio_distance))))
            sum_of_cents_voice += abs(ratio_to_cents(ratio_distance))
            # print(f' {prev_note}, {current_note} '
            #       f'{all_ratio_strings[prev_note]} to {all_ratio_strings[current_note]} distance ratio: ' 
            #       f' {Fraction(ratio_distance).limit_denominator(100)} decimal ratio: '
            #       f' {round(ratio_distance,3)} cents: '
            #       f' {ratio_to_cents(ratio_distance)}')
            # 
            #                        value contour:  ____/⎺⎺\____ 
            #                                                    ⎺⎺⎺⎺\__/⎺⎺⎺⎺
            #                    from where you are, take 128 to reach new ratio stay at that ratio for 128
            fn_array = np.append(fn_array, np.array([each_f_step, ratio_distance, each_f_step, ratio_distance]), axis = None)
            total_f_steps += each_f_step * 2
            prev_note = current_note
    # print(f'{total_f_steps = }')
    
    print(f'{round(sum_of_cents_voice,1) = }, {round(max_cents_per_voice,2) = }')
    return fn_array

In [206]:
# slide bewteen any number of arbitrary chords definded by mode, root, rank, and inversion
def multiple_chord_slide(note_dict):
    
    stored_gliss_table, current_gliss_table = dmu.retrieve_gliss_tables() # retrieve the existing glissando tables from dmu
    print(f'{stored_gliss_table.shape = }')
    print(f'{current_gliss_table = }')
    # assemble a numpy array of all the hexad chords in bridge_chords from the bridge_chords dictionary. Optimize the inversions
    # We previously built 20 different 9 chord array of hexads with inversions based on the path through the hexad that minizes the slides
    chosen_array = note_dict["bridge_chord_array"]
    array_of_chords = best_bridge_chord_array[chosen_array]
    print(f'{chosen_array = }, {array_of_chords}') # print the chosen array nine notes and four voices
    # collect the four arrays of gliss tables through the nine chords
    voice_count = 4
    gliss_f_table = np.array([build_voice_slide(current_gliss_table + inx, array_of_chords[inx]) for inx in np.arange(voice_count)])
    current_gliss_table = current_gliss_table + voice_count
    # print(f'{gliss_f_table[:,0] = }') # array([800., 801., 802., 803.])
    # print(f'{gliss_f_table.shape = }') # (4,37)
    pad_size = 70 - gliss_f_table.shape[1] # 33 zeros needed to pad the gliss table.
    pad_gliss = np.zeros((4, pad_size), dtype = float)
    gliss_f_table = np.concatenate((gliss_f_table, pad_gliss), axis = 1) # is this the right axis?
    # print(f'{gliss_f_table.shape = }, {pad_gliss.shape = }')
    stored_gliss_table = np.concatenate((stored_gliss_table, gliss_f_table)) # this is a global variable.
    # print(f'{stored_gliss_table.shape = }') # (4,70) (4,33)
    gliss = gliss_f_table[:,0]
    # print(f'{gliss.shape = }') # (4,)
    current_gliss_table = dmu.update_gliss_table(stored_gliss_table, current_gliss_table)
    # print(f'{current_gliss_table = }') # 804
    
    octv = note_dict["octv"] # if it's an array, then iterate over it, after setting duration to zero for the first iterations, then the final iteration gets the duration value
    repeats = octv.shape[0]
    tones_1 = array_of_chords.T[0,:] # just the first note for each instrument
    print(f'in multiple_chord_slide. {tones_1 = }')
    # The number of notes is 4. The number of instruments may be something else. In that case we need to duplicate some notes to reach the number of instruments
    num_inst = note_dict["instruments"].shape[0]
    num_notes = 4
    while num_notes < num_inst:
        # print(f'{num_inst = }, {num_notes = }, {tones_1.shape = }')
        # copy the duplicate the first element (0) to be the last element until the number of notes = number of instruments
        tones_1 = np.append(tones_1, tones_1[0], axis = None)
        # octave = np.append(octave, octave[0], axis = None)
        gliss = np.append(gliss, gliss[0], axis = None)
        num_notes += 1
    if num_inst < num_notes:
        print(f'Not enough instruments specified in {note_dict["instruments"] = }, {num_notes = }')
        return None
    dur = np.ones((repeats,), dtype = float) * note_dict["dur"] # all the notes have the same duration 
    dur[:-1] = 0 # set the first iterations to 0 duration , last one stays at the normal duration.
    hold = np.ones((num_inst,), dtype = float) * note_dict["hold"] # how long to hold them
    env = np.ones((num_inst,), dtype = float) * note_dict["env"] # what envelope to use for all of them
    renv = np.ones((num_inst,), dtype = float) * note_dict["renv"] # what should the right envelope be
    # Ins Sta Dur Vel Ton Oct Voi Ste En1 Gls Ups Ren 2gl  3gl Vol
    # [1,  0, 4,  60, 32,  2, 7,  8,  1, 800, 1,  1,  799, 799, 30]	start: 0.0,	dur: 4.4
    # the goal is to play notes in each instrument with duration = 0 until you play the last octave, then set duration to 4
    iterations = 0
    instruments = np.empty((num_inst * repeats, 15), dtype = float)
    
    for inst in np.arange(num_inst): # once for each instrument
        # print(f'once for each instrument {inst = }')
        for octave in octv: # number of repeats = number of elements in the octv array
            # print(f'once for each octv {octave = }')
            current_octave = iterations % repeats
            # print(f'{current_octave = }, {iterations = }')
            instruments[iterations] = instrument(tones_1[inst], octave, dur[current_octave], hold[inst], gliss[inst], \
                        note_dict["instruments"][inst], note_dict["defaults"][1], note_dict["defaults"][2], env[inst], \
                        note_dict["defaults"][3], renv[inst], note_dict["defaults"][4], note_dict["defaults"][5], \
                        note_dict["defaults"][6])
            iterations += 1
            
    return instruments

In [207]:
# play all the notes and voices in the (4,9) array
def note_array(note_dict):
    # load the local array from the dictionary value
    tones_1 = note_dict["note_array"]
    print(f'{tones_1.shape = }') # tones_1.shape = (4, 9)
           
    num_inst = note_dict["instruments"].shape[0]
    num_notes = tones_1.shape[0]
    print(f'{num_inst = }, {num_notes = }') # num_inst = 5, num_notes = 4
    print(f'{tones_1.shape = }, {tones_1[0].reshape(1,9).shape = }')
    while num_notes < num_inst:
        tones_1 = np.concatenate((tones_1, tones_1[0].reshape(1,9)), axis = 0)
        num_notes = tones_1.shape[0]
        print(f'after appending {num_notes = }')
    if num_inst < num_notes:
        print(f'Not enough instruments specified in {note_dict["instruments"] = }, {num_notes = }')
        return None
  
    dur = np.ones((num_inst,), dtype = float) * note_dict["dur"] # all the notes have the same duration - I'll have to fix this later
    hold = np.ones((num_inst,), dtype = float) * note_dict["hold"] # how long to hold them - need separate holds for each chord
    env = np.ones((num_inst,), dtype = float) * note_dict["env"] # what envelope to use for all of them
    renv = np.ones((num_inst,), dtype = float) * note_dict["renv"] # what should the right envelope be
    instruments = np.empty((tones_1.shape[0] * tones_1.shape[1] , 15), dtype = float)
    print(f'{instruments.shape = }') # 
    
    iterations = 0
    for notes in tones_1.T:
        print(f'{notes.shape = }, {notes = }') # notes.shape = (4,), notes = array([40, 44, 32, 36])
        octave = dmu.build_scale_mask(notes) + note_dict["octv"] # get the octave mask
        print(f'{dmu.build_scale_mask(notes) = }')
        for inst in np.arange(num_inst):
            
        # (tone, octave, duration, hold, gliss, voice_name, velocity, stereo, env, upsample, renv, gliss2, gliss3, volume):
            instruments[iterations] = instrument(notes[inst], octave[inst], dur[inst], hold[inst], note_dict["defaults"][0], \
                            note_dict["instruments"][inst], note_dict["defaults"][1], note_dict["defaults"][2], env[inst], \
                            note_dict["defaults"][3], renv[inst], note_dict["defaults"][4], note_dict["defaults"][5], \
                            note_dict["defaults"][6])
            iterations += 1
    print(f'{instruments.shape = }')
    return instruments

In [251]:
# reload(dmu)
init_voice_start_times()
dmu.init_stored_gliss()
stored_gliss = dmu.init_stored_gliss() # resets the global glissando array and the global current_gliss_table variable to 800
winds = np.empty((0,15), dtype = float)
woodwinds = np.array(["obo","cla","bfl","bss","frn"])
defaults = np.array([799, 60, 8, 1, 799, 799, 30]) # flat gliss #, velocity, stereo, envelope, 2 flat gliss, volume
octv = np.array([3]) # the shape of the array dictates the number of repeats to achieve all slides across all octaves provided
note_dict = {0: {'exec': multiple_chord_slide, 'dur': 0, 'hold': 45, 'octv': octv, 'defaults': defaults, 
              'env': 1, 'renv': 1, 'instruments': woodwinds, "bridge_chord_array":  rng.integers(best_bridge_chord_array.shape[0])},
    
             2: {'exec': multiple_chord_slide, 'dur': 45, 'hold': 45, 'octv': octv, 'defaults': defaults, 
              'env': 1, 'renv': 1, 'instruments': woodwinds, "bridge_chord_array":  rng.integers(best_bridge_chord_array.shape[0])},
            }

             
# rng.integers(best_bridge_chord_array.shape[0])
# winds = note_dict['exec'](note_dict)
for ind in note_dict:
    winds = np.concatenate((winds, note_dict[ind]["exec"](note_dict[ind])),axis = 0)

fixed_winds = fix_start_times(winds) # fix the start times based on durations.    
# print(f'{winds = }\n{fixed_winds = }')

stored_gliss_table.shape = (0, 70)
current_gliss_table = 800
chosen_array = 8, [[ 40 196 130 232 128   8  76  78  44]
 [ 44 200  66 236  64  12  12  14  32]
 [ 32 204   2 224   0   0 204 206  36]
 [ 36 192 194 228 192   4 140 142  40]]
round(sum_of_cents_voice,1) = 1136.9, round(max_cents_per_voice,2) = 315.6
round(sum_of_cents_voice,1) = 949.3, round(max_cents_per_voice,2) = 266.9
round(sum_of_cents_voice,1) = 794.1, round(max_cents_per_voice,2) = 203.9
round(sum_of_cents_voice,1) = 1088.2, round(max_cents_per_voice,2) = 266.9
stored_gliss.shape = (4, 70), current_gliss_table = 804
in multiple_chord_slide. tones_1 = array([40, 44, 32, 36])
stored_gliss_table.shape = (4, 70)
current_gliss_table = 804
chosen_array = 9, [[ 32 200 130 232  64   8  76 142  40]
 [ 36 204  66 236   0  12  12  78  44]
 [ 40 192   2 224 192   0 204  14  32]
 [ 44 196 194 228 128   4 140 206  36]]
round(sum_of_cents_voice,1) = 944.9, round(max_cents_per_voice,2) = 231.2
round(sum_of_cents_voice,1) = 1108.3, rou

In [245]:
# reload(dmu)
init_voice_start_times()
dmu.init_stored_gliss()
stored_gliss = dmu.init_stored_gliss() # resets the global glissando array and the global current_gliss_table variable to 800
winds = np.empty((0,15), dtype = float)
woodwinds = np.array(["obo","cla","bfl","bss","frn"])
defaults = np.array([799, 60, 8, 1, 799, 799, 30]) # flat gliss #, velocity, stereo, envelope, 2 flat gliss, volume
octv = 3 # the shape of the array dictates the number of repeats to achieve all slides across all octaves provided
note_dict = {0:  {'exec': note_array, 'dur': 0, 'hold': 1, 'octv': octv, 'defaults': defaults, 
              'env': 1, 'renv': 1, 'instruments': woodwinds, 
             "note_array":  best_bridge_chord_array[rng.integers(best_bridge_chord_array.shape[0])] }, 
             1: {'exec': note_array, 'dur': 1, 'hold': 1, 'octv': octv, 'defaults': defaults, 
              'env': 1, 'renv': 1, 'instruments': woodwinds, 
             "note_array":  best_bridge_chord_array[rng.integers(best_bridge_chord_array.shape[0])] },
             
            }
# rng.integers(best_bridge_chord_array.shape[0])
# winds = note_dict['exec'](note_dict)
for ind in note_dict:
    winds = np.concatenate((winds, note_dict[ind]["exec"](note_dict[ind])),axis = 0)
fixed_winds = fix_start_times(winds) # fix the start times based on durations.    
# print(f'{winds = }\n{fixed_winds = }')

tones_1.shape = (4, 9)
num_inst = 5, num_notes = 4
tones_1.shape = (4, 9), tones_1[0].reshape(1,9).shape = (1, 9)
after appending num_notes = 5
instruments.shape = (45, 15)
notes.shape = (5,), notes = array([44, 32, 36, 40, 44])
dmu.build_scale_mask(notes) = array([0, 0, 1, 1, 1])
notes.shape = (5,), notes = array([196, 200, 204, 192, 196])
dmu.build_scale_mask(notes) = array([0, 0, 1, 1, 1])
notes.shape = (5,), notes = array([130,  66,   2, 194, 130])
dmu.build_scale_mask(notes) = array([0, 0, 1, 1, 1])
notes.shape = (5,), notes = array([232, 236, 224, 228, 232])
dmu.build_scale_mask(notes) = array([0, 0, 1, 1, 1])
notes.shape = (5,), notes = array([ 64,   0, 192, 128,  64])
dmu.build_scale_mask(notes) = array([0, 1, 1, 1, 1])
notes.shape = (5,), notes = array([ 8, 12,  0,  4,  8])
dmu.build_scale_mask(notes) = array([0, 0, 1, 1, 1])
notes.shape = (5,), notes = array([ 76,  12, 204, 140,  76])
dmu.build_scale_mask(notes) = array([0, 0, 1, 1, 1])
notes.shape = (5,), notes = array([142,

In [252]:
a, b = start_csound()
if a is not None:
    cs = a
    pt = b

csound is already running


In [253]:
print(f'glide ratios for all the slides in the whole piece:')
first_rows_only = False
inx = 0
for row in dmu.retrieve_gliss_tables()[0]: # pass some f tables for glissandi to the instance of csound
    pt.scoreEvent(0, 'f', row)
    inx += 1
    if first_rows_only:
        if inx < 5: 
            print(f'{[round(item,3) for item in row[:37]]}')
    else: 
        print(f'{[round(item,3) for item in row[:37]]}')
inx = 0        
print(f'here are the notes sent to csound')
print(f'InsSta Dur VelTon OctVoiSteEn1 Gls UpsRen 2gl  3gl Vol')
for row in fixed_winds: # send the note information to csound
    pt.scoreEvent(0, 'i', row)
    inx += 1
    if first_rows_only:
        if inx < 5:
            print(f'{[int(row[item]) for item in np.arange(15)]}', end = '\t') # print the note values
            print(f'start: {round(row[1],3)},\tdur: {round(row[2],3)}')
    else:
        print(f'{[int(row[item]) for item in np.arange(15)]}', end = '\t') # print the note values
        print(f'start: {round(row[1],3)},\tdur: {round(row[2],3)}')

glide ratios for all the slides in the whole piece:
[800.0, 0.0, 2048.0, -7.0, 1.0, 128.0, 1.071, 128.0, 1.071, 128.0, 1.05, 128.0, 1.05, 128.0, 1.067, 128.0, 1.067, 128.0, 0.833, 128.0, 0.833, 128.0, 1.125, 128.0, 1.125, 128.0, 0.933, 128.0, 0.933, 128.0, 1.071, 128.0, 1.071, 128.0, 1.037, 128.0, 1.037]
[801.0, 0.0, 2048.0, -7.0, 1.0, 128.0, 1.102, 128.0, 1.102, 128.0, 1.05, 128.0, 1.05, 128.0, 1.037, 128.0, 1.037, 128.0, 0.857, 128.0, 0.857, 128.0, 1.094, 128.0, 1.094, 128.0, 1.0, 128.0, 1.0, 128.0, 1.071, 128.0, 1.071, 128.0, 0.948, 128.0, 0.948]
[802.0, 0.0, 2048.0, -7.0, 1.0, 128.0, 1.125, 128.0, 1.125, 128.0, 1.125, 128.0, 1.125, 128.0, 0.948, 128.0, 0.948, 128.0, 0.938, 128.0, 0.938, 128.0, 1.0, 128.0, 1.0, 128.0, 1.0, 128.0, 1.0, 128.0, 1.071, 128.0, 1.071, 128.0, 1.037, 128.0, 1.037]
[803.0, 0.0, 2048.0, -7.0, 1.0, 128.0, 1.029, 128.0, 1.029, 128.0, 1.125, 128.0, 1.125, 128.0, 1.037, 128.0, 1.037, 128.0, 0.857, 128.0, 0.857, 128.0, 1.094, 128.0, 1.094, 128.0, 0.933, 128.0, 0.9

In [213]:
stop_csound(cs, pt)

0dBFS level = 32768.0
--Csound version 6.16 (double samples) Aug 10 2021
[commit: none]
libsndfile-1.0.31
